## Imports

In [69]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


# Files Upload

In [70]:
df_awards_players = pd.read_csv('data/awards_players.csv')
df_coaches = pd.read_csv('data/coaches.csv')
df_players_teams = pd.read_csv('data/players_teams.csv')
df_players = pd.read_csv('data/players.csv')
df_series_post = pd.read_csv('data/series_post.csv')
df_teams = pd.read_csv('data/teams.csv')
df_teams_post = pd.read_csv('data/teams_post.csv')

## Data Cleanup and analysis

## Drop columns

Drop collumns that have null values or that have all the same value

In [71]:

def drop_columns_with_all_nan(df):
    # Step 1: Check for columns with all NaN values
    columns_to_drop = []
    for col in df.columns:
        unique_values = df[col].unique()
        if len(unique_values) == 1 or all(pd.isna(x) for x in unique_values):
            columns_to_drop.append(col)

    # Step 2: Drop identified columns
    df.drop(columns=columns_to_drop, inplace=True)

    # Step 3: Print the identified columns to be dropped
    print(columns_to_drop)




In [72]:
drop_columns_with_all_nan(df_teams)
print(df_teams.duplicated().sum())
df_teams.drop_duplicates(inplace=True)

drop_columns_with_all_nan(df_teams_post)
print(df_teams_post.duplicated().sum())
df_teams_post.drop_duplicates(inplace=True)

drop_columns_with_all_nan(df_series_post)
print(df_series_post.duplicated().sum())
df_series_post.drop_duplicates(inplace=True)

drop_columns_with_all_nan(df_players)
print(df_players.duplicated().sum())
df_players.drop_duplicates(inplace=True)

drop_columns_with_all_nan(df_players_teams)
print(df_players_teams.duplicated().sum())
df_players_teams.drop_duplicates(inplace=True)

drop_columns_with_all_nan(df_coaches)
print(df_coaches.duplicated().sum())
print(df_coaches.duplicated(subset=['tmID', 'year']).sum())
df_coaches.drop_duplicates(inplace=True)

drop_columns_with_all_nan(df_awards_players)
print(df_awards_players.duplicated().sum())
df_awards_players.drop_duplicates(inplace=True)



['lgID', 'divID', 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB']
0
['lgID']
0
['lgIDWinner', 'lgIDLoser']
0
['firstseason', 'lastseason']
0
['lgID']
0
['lgID']
0
20
['lgID']
0


## Agregate data according with previous years

As we have the information about the year at the end of the playoff in each dataset, to avoid data leakage we will aggregate the data from the two previous years to the current year. And create new datasets by doing so.

In [73]:
years_back=2

### Teams

As there are new teams every year we decided to drop all the collumns as only the teams that played before had any relevant information. So as to not have any bias towards the teams that played before, all collumns were dropped.

In [74]:
# Replace 'N' and 'Y' in the 'playoff' column with 0 and 1
df_teams['playoff'] = df_teams['playoff'].map({'N': 0, 'Y': 1})
df_teams.sort_values(by=['year', 'tmID'], ascending=[True, True], inplace=True)

years = df_teams['year'].unique()
teams = df_teams['tmID'].unique()

df_teams_copy = df_teams.copy()

for year in years:
    for team in teams:
        df_teams_copy.loc[(df_teams_copy['year'] == year) & (df_teams_copy['tmID'] == team), 'num_playoff_appearances'] = df_teams[(df_teams['year'] >= (year - years_back)) & (df_teams['year'] < year) & (df_teams['tmID'] == team)]['playoff'].sum()
        df_teams_copy.loc[(df_teams_copy['year'] == year) & (df_teams_copy['tmID'] == team), 'total_first_round_won'] = (df_teams[(df_teams['year'] >= (year - years_back)) &(df_teams['year'] < year) & (df_teams['tmID'] == team)]['firstRound'] == 'W').sum()
        df_teams_copy.loc[(df_teams_copy['year'] == year) & (df_teams_copy['tmID'] == team), 'total_first_round_lost'] = (df_teams[(df_teams['year'] >= (year - years_back)) &(df_teams['year'] < year) & (df_teams['tmID'] == team)]['firstRound'] == 'L').sum()
        df_teams_copy.loc[(df_teams_copy['year'] == year) & (df_teams_copy['tmID'] == team), 'total_semis_won'] = (df_teams[(df_teams['year'] >= (year - years_back)) &(df_teams['year'] < year) & (df_teams['tmID'] == team)]['semis'] == 'W').sum()
        df_teams_copy.loc[(df_teams_copy['year'] == year) & (df_teams_copy['tmID'] == team), 'total_semis_lost'] = (df_teams[(df_teams['year'] >= (year - years_back)) &(df_teams['year'] < year) & (df_teams['tmID'] == team)]['semis'] == 'L').sum()
        df_teams_copy.loc[(df_teams_copy['year'] == year) & (df_teams_copy['tmID'] == team), 'total_finals_won'] = (df_teams[(df_teams['year'] >= (year - years_back)) &(df_teams['year'] < year) & (df_teams['tmID'] == team)]['finals'] == 'W').sum()
        df_teams_copy.loc[(df_teams_copy['year'] == year) & (df_teams_copy['tmID'] == team), 'total_finals_lost'] = (df_teams[(df_teams['year'] >= (year - years_back)) &(df_teams['year'] < year) & (df_teams['tmID'] == team)]['finals'] == 'L').sum()
        df_teams_copy.loc[(df_teams_copy['year'] == year) & (df_teams_copy['tmID'] == team), 'mean_won'] = (df_teams[(df_teams['year'] >= (year - years_back)) &(df_teams['year'] < year) & (df_teams['tmID'] == team)]['won']).mean()
        df_teams_copy.loc[(df_teams_copy['year'] == year) & (df_teams_copy['tmID'] == team), 'mean_lost'] = (df_teams[(df_teams['year'] >= (year - years_back)) &(df_teams['year'] < year) & (df_teams['tmID'] == team)]['lost']).mean()
        df_teams_copy.loc[(df_teams_copy['year'] == year) & (df_teams_copy['tmID'] == team), 'rank'] = (df_teams[(df_teams['year'] >= (year - years_back)) &(df_teams['year'] < year) & (df_teams['tmID'] == team)]['rank']).mean()
         
df_teams_copy.drop(columns=['firstRound', 'semis', 'finals',"won","lost", 'franchID', 'name', 'arena', "o_fgm","o_fga","o_ftm","o_fta","o_3pm","o_3pa","o_oreb","o_dreb","o_reb","o_asts","o_pf","o_stl","o_to","o_blk","o_pts","d_fgm","d_fga","d_ftm","d_fta","d_3pm","d_3pa","d_oreb","d_dreb","d_reb","d_asts","d_pf","d_stl","d_to","d_blk","d_pts","GP","homeW","homeL","awayW","awayL","confW","confL","min","attend"], inplace=True)
df_teams_copy.fillna(0, inplace=True)   
df_teams_copy.to_csv('data/teams_processed.csv', index=False)

encoder = OneHotEncoder()
categorical_features = ['confID']
for feature in categorical_features:
    onehotarray = encoder.fit_transform(df_teams_copy[[feature]]).toarray()
    items = [f'{feature}_{item}' for item in encoder.categories_[0]]
    df_teams_copy[items] = onehotarray
df_teams_copy=df_teams_copy.drop(categorical_features, axis=1)

df_teams_copy.to_csv('data/teams_processed.csv', index=False)

### Teams post

In [75]:
df_teams_post_copy = df_teams_post.copy()
sum_collumns = ["W", "L"]
for year in years:
    for team in teams:
        for column in sum_collumns:
            df_teams_post_copy.loc[(df_teams_post_copy['year'] == year) & (df_teams_post_copy['tmID'] == team), column] = df_teams_post[(df_teams_post['year'] >= (year - years_back)) &(df_teams_post['year'] < year) & (df_teams_post['tmID'] == team)][column].sum()

df_teams_post_copy.fillna(0, inplace=True)
df_teams_post_copy.to_csv('data/teams_post_processed.csv', index=False)   


### series post

For each team some the number of win and losses they had in the year. For each year we then agregate the information from the previous two years.

In [76]:
winning_team_stats = df_series_post.groupby(["year", "tmIDWinner"])[["W"]].sum().reset_index()
winning_team_stats.columns = ["year", "tmID", "total_wins"]

losing_team_stats = df_series_post.groupby(["year", "tmIDLoser"])[["L"]].sum().reset_index()
losing_team_stats.columns = ["year", "tmID", "total_losses"]

# Merge the winning and losing team statistics
team_stats = winning_team_stats.merge(losing_team_stats, on=["year", "tmID"], how="outer").fillna(0)
team_stats_copy = team_stats.copy()
average_collumns = ["total_wins", "total_losses"]
for year in years:
    for team in teams:
        for column in average_collumns:
            team_stats_copy.loc[(team_stats_copy['year'] == year) & (team_stats_copy['tmID'] == team), column] = team_stats[(team_stats['year'] >= (year - years_back)) &(team_stats['year'] < year) & (team_stats['tmID'] == team)][column].mean()
team_stats_copy.fillna(0, inplace=True)
team_stats_copy.to_csv('data/series_post_processed.csv', index=False)

### Players teams

For each player calculate the statistics of the previous two years, independently of the team.

In [77]:
df_players_teams_copy = df_players_teams.copy()

average_collumns = ["GP","GS","minutes","points","oRebounds","dRebounds","rebounds","assists","steals","blocks","turnovers","PF","fgAttempted","fgMade","ftAttempted","ftMade","threeAttempted","threeMade","dq","PostGP","PostGS","PostMinutes","PostPoints","PostoRebounds","PostdRebounds","PostRebounds","PostAssists","PostSteals","PostBlocks","PostTurnovers","PostPF","PostfgAttempted","PostfgMade","PostftAttempted","PostftMade","PostthreeAttempted","PostthreeMade","PostDQ"]
for year in years:
    for team in teams:
        players = df_players_teams[(df_players_teams['year'] == year) & (df_players_teams['tmID'] == team)]['playerID'].unique()
        for player in players:
            for column in average_collumns:
                #df_players_teams_copy.loc[(df_players_teams_copy['year'] == year) & (df_players_teams_copy['tmID'] == team) & (df_players_teams_copy['playerID'] == player), column] = df_players_teams[(df_players_teams['year'] < year) & (df_players_teams['tmID'] == team) & (df_players_teams['playerID'] == player)][column].mean()
                df_players_teams_copy.loc[(df_players_teams_copy['year'] == year) & (df_players_teams_copy['tmID'] == team) & (df_players_teams_copy['playerID'] == player), column] = df_players_teams[(df_players_teams['year'] >= (year - years_back)) &(df_players_teams['year'] < year)  & (df_players_teams['playerID'] == player)][column].mean()

df_players_teams_copy.dropna(inplace=True)
df_players_teams_copy.drop(columns=['stint'], inplace=True)
df_players_teams_copy.to_csv('data/players_teams_processed.csv', index=False)

print(df_players_teams.info(), df_players_teams_copy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1876 entries, 0 to 1875
Data columns (total 42 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   playerID            1876 non-null   object
 1   year                1876 non-null   int64 
 2   stint               1876 non-null   int64 
 3   tmID                1876 non-null   object
 4   GP                  1876 non-null   int64 
 5   GS                  1876 non-null   int64 
 6   minutes             1876 non-null   int64 
 7   points              1876 non-null   int64 
 8   oRebounds           1876 non-null   int64 
 9   dRebounds           1876 non-null   int64 
 10  rebounds            1876 non-null   int64 
 11  assists             1876 non-null   int64 
 12  steals              1876 non-null   int64 
 13  blocks              1876 non-null   int64 
 14  turnovers           1876 non-null   int64 
 15  PF                  1876 non-null   int64 
 16  fgAttempted         1876

### Coaches

In [78]:
df_coaches_copy = df_coaches.copy()

average_collumns = ["won","lost","post_wins","post_losses"]
for year in years:
    for team in teams:
        coaches= df_coaches[(df_coaches['year'] == year) & (df_coaches['tmID'] == team)]['coachID'].unique()
        for coach in coaches:
            for column in average_collumns:
                df_coaches_copy.loc[(df_coaches_copy['year'] == year) & (df_coaches_copy['tmID'] == team) & (df_coaches_copy['coachID'] == coach), column] = df_coaches[(df_coaches['year'] >= (year - years_back)) &(df_coaches['year'] < year) & (df_coaches['coachID'] == coach)][column].mean()         

df_coaches_copy.fillna(0, inplace=True)
df_coaches_copy.to_csv('data/coaches_processed.csv', index=False)


### Players

Drop players from whom we have no information

In [79]:
df_players = df_players[~((df_players['pos'].isna()) &
                            (df_players['height'] == 0.0) &
                            (df_players['weight'] == 0) &
                            (df_players['college'].isna()) &
                            (df_players['collegeOther'].isna()) &
                            (df_players['birthDate'] == "0000-00-00") &
                            (df_players['deathDate'] == "0000-00-00"))]
df_players.to_csv('data/players_processed.csv', index=False)

### Awards

In [80]:
award_counts = df_awards_players.groupby(["year", "playerID"]).size().reset_index(name="award_count")

# Create an empty DataFrame to store the results
award_counts_copy = pd.DataFrame(columns=["year", "playerID", "award_count"])
players = df_awards_players['playerID'].unique()
# Iterate over all possible combinations of "year" and "player" and add them to the DataFrame
for year in years:
    for player in players:
        award_count = award_counts[(award_counts['year'] >= (year - years_back)) & (award_counts['year'] < year) & (award_counts['playerID'] == player)]['award_count'].sum()
        new_row = {"year": year, "playerID": player, "award_count": award_count}
        award_counts_copy = pd.concat([award_counts_copy, pd.DataFrame([new_row])], ignore_index=True)

award_counts_copy.dropna(inplace=True)
award_counts_copy.to_csv('data/awards_players_processed.csv', index=False)